# 7-2 Join one-to-many tables

## Add sales and reservations of the hotel to hotel-master

### Pandas

In [1]:
import pandas as pd

In [2]:
pd_reservation = pd.read_parquet("../../data/reservation.parquet")
pd_hotel       = pd.read_parquet("../../data/hotel.parquet")

In [3]:
pd_reservation.head()

,reservation_id,hotel_id,customer_id,reserved_at,checkin_date,checkout_date,length_of_stay,people_num,total_price,status,canceled_at
0,1,2460,53431,2013-12-31 07:00:14+09:00,2014-12-31 00:00:00+09:00,2015-01-03 00:00:00+09:00,3,2,37800,reserved,NaT
1,2,962,488390,2013-12-31 08:23:35+09:00,2014-12-31 00:00:00+09:00,2015-01-02 00:00:00+09:00,2,3,42000,reserved,NaT
2,3,558,341335,2013-12-31 09:02:05+09:00,2014-12-31 00:00:00+09:00,2015-01-01 00:00:00+09:00,1,2,20400,reserved,NaT
3,4,3666,398981,2013-12-31 23:44:54+09:00,2014-12-31 00:00:00+09:00,2015-01-01 00:00:00+09:00,1,4,39600,reserved,NaT
4,5,2180,220381,2014-01-01 02:47:50+09:00,2014-12-31 00:00:00+09:00,2015-01-01 00:00:00+09:00,1,3,16500,reserved,NaT


In [4]:
pd_hotel.head()

,hotel_id,hotel_name,hotel_type,address_prefecture,address_city,address_town,address_zipcode,unit_price,user_rating,tag_001,...,tag_021,tag_022,tag_023,tag_024,tag_025,tag_026,tag_027,tag_028,tag_029,tag_030
0,1,北飯岡ホテル,ビジネスホテル,岩手県,盛岡市,北飯岡,020-0857,10300,3.256591,0,...,0,1,<NA>,0,0,1,0,0,0,0
1,2,西二条南温泉ホテル,リゾートホテル,北海道,中川郡美深町,西二条南,098-2242,6800,3.453642,0,...,0,0,0,<NA>,1,0,0,0,0,0
2,3,小屋敷ペンション,民宿,青森県,黒石市,小屋敷,036-0511,18600,1.381796,<NA>,...,0,0,0,1,<NA>,<NA>,0,0,0,0
3,4,中後町民宿,民宿,愛知県,碧南市,中後町,447-0042,4900,2.090353,0,...,0,0,0,0,0,0,0,0,1,0
4,5,鵜沼台ホテル,ビジネスホテル,岐阜県,各務原市,鵜沼台,509-0121,9000,2.771490,0,...,0,0,0,0,0,1,1,1,0,0


In [5]:
(
    pd_hotel
    .merge(
        pd_reservation
        .query("status != 'canceled' and checkout_date.dt.year == 2019")
        .groupby("hotel_id")
        .total_price
        .agg(["sum", "size"]),
        on="hotel_id",
        how="left",
    )
    .fillna({"sum": 0, "size": 0})
)

,hotel_id,hotel_name,hotel_type,address_prefecture,address_city,address_town,address_zipcode,unit_price,user_rating,tag_001,...,tag_023,tag_024,tag_025,tag_026,tag_027,tag_028,tag_029,tag_030,sum,size
0,1,北飯岡ホテル,ビジネスホテル,岩手県,盛岡市,北飯岡,020-0857,10300,3.256591,0,...,<NA>,0,0,1,0,0,0,0,3017900.0,83.0
1,2,西二条南温泉ホテル,リゾートホテル,北海道,中川郡美深町,西二条南,098-2242,6800,3.453642,0,...,0,<NA>,1,0,0,0,0,0,0.0,0.0
2,3,小屋敷ペンション,民宿,青森県,黒石市,小屋敷,036-0511,18600,1.381796,<NA>,...,0,1,<NA>,<NA>,0,0,0,0,10248600.0,143.0
3,4,中後町民宿,民宿,愛知県,碧南市,中後町,447-0042,4900,2.090353,0,...,0,0,0,0,0,0,1,0,1808100.0,94.0
4,5,鵜沼台ホテル,ビジネスホテル,岐阜県,各務原市,鵜沼台,509-0121,9000,2.771490,0,...,0,0,0,1,1,1,0,0,2421000.0,73.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,田辺通旅館,旅館,愛知県,名古屋市瑞穂区,田辺通,467-0027,12100,NaN,0,...,0,0,0,0,<NA>,0,1,0,6957500.0,136.0
4996,4997,六本木温泉旅館,旅館,東京都,港区,六本木,106-8544,8800,1.499441,<NA>,...,0,0,0,0,1,0,0,0,1663200.0,57.0
4997,4998,丸山吉野町温泉旅館,旅館,福岡県,北九州市門司区,丸山吉野町,801-0824,15700,2.336651,0,...,0,0,1,0,0,0,<NA>,1,5275200.0,81.0
4998,4999,神埼郡吉野ヶ里町ビジネスホテル,ビジネスホテル,佐賀県,神埼郡吉野ヶ里町,<NA>,842-0000,25300,1.559659,<NA>,...,<NA>,0,0,0,0,0,<NA>,0,12042800.0,106.0


### Polars

In [6]:
import polars as pl

In [7]:
pl_reservation = pl.read_parquet("../../data/reservation.parquet")
pl_hotel       = pl.read_parquet("../../data/hotel.parquet")

In [8]:
pl_reservation.head()

reservation_id,hotel_id,customer_id,reserved_at,checkin_date,checkout_date,length_of_stay,people_num,total_price,status,canceled_at
i64,i64,i64,"datetime[ns, Asia/Tokyo]","datetime[ns, Asia/Tokyo]","datetime[ns, Asia/Tokyo]",i64,i64,i64,str,"datetime[ns, Asia/Tokyo]"
1,2460,53431,2013-12-31 07:00:14 JST,2014-12-31 00:00:00 JST,2015-01-03 00:00:00 JST,3,2,37800,"""reserved""",null
2,962,488390,2013-12-31 08:23:35 JST,2014-12-31 00:00:00 JST,2015-01-02 00:00:00 JST,2,3,42000,"""reserved""",null
3,558,341335,2013-12-31 09:02:05 JST,2014-12-31 00:00:00 JST,2015-01-01 00:00:00 JST,1,2,20400,"""reserved""",null
4,3666,398981,2013-12-31 23:44:54 JST,2014-12-31 00:00:00 JST,2015-01-01 00:00:00 JST,1,4,39600,"""reserved""",null
5,2180,220381,2014-01-01 02:47:50 JST,2014-12-31 00:00:00 JST,2015-01-01 00:00:00 JST,1,3,16500,"""reserved""",null


In [9]:
pl_hotel.head()

hotel_id,hotel_name,hotel_type,address_prefecture,address_city,address_town,address_zipcode,unit_price,user_rating,tag_001,tag_002,tag_003,tag_004,tag_005,tag_006,tag_007,tag_008,tag_009,tag_010,tag_011,tag_012,tag_013,tag_014,tag_015,tag_016,tag_017,tag_018,tag_019,tag_020,tag_021,tag_022,tag_023,tag_024,tag_025,tag_026,tag_027,tag_028,tag_029,tag_030
i64,str,str,str,str,str,str,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
1,"""北飯岡ホテル""","""ビジネスホテル""","""岩手県""","""盛岡市""","""北飯岡""","""020-0857""",10300,3.256591,0,0,0,0,0,0,0,0,0,0,0,0,0,1,null,0,0,1,1,0,0,1,null,0,0,1,0,0,0,0
2,"""西二条南温泉ホテル""","""リゾートホテル""","""北海道""","""中川郡美深町""","""西二条南""","""098-2242""",6800,3.453642,0,0,0,null,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,null,1,0,0,0,0,0
3,"""小屋敷ペンション""","""民宿""","""青森県""","""黒石市""","""小屋敷""","""036-0511""",18600,1.381796,null,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null,0,0,0,1,null,null,0,0,0,0
4,"""中後町民宿""","""民宿""","""愛知県""","""碧南市""","""中後町""","""447-0042""",4900,2.090353,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
5,"""鵜沼台ホテル""","""ビジネスホテル""","""岐阜県""","""各務原市""","""鵜沼台""","""509-0121""",9000,2.77149,0,0,0,1,0,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0


In [10]:
(
    pl_hotel
    .join(
        pl_reservation
        .filter(pl.col("status") != "canceled")
        .filter(pl.col("checkout_date").dt.year() == 2019)
        .group_by("hotel_id")
        .agg([
            pl.col("total_price").sum(),
            pl.len()
        ]),
        on="hotel_id",
        how="left"
    )
    .with_columns(
        total_price=pl.col("total_price").fill_null(0),
        len=pl.col("len").fill_null(0)
    )
)

hotel_id,hotel_name,hotel_type,address_prefecture,address_city,address_town,address_zipcode,unit_price,user_rating,tag_001,tag_002,tag_003,tag_004,tag_005,tag_006,tag_007,tag_008,tag_009,tag_010,tag_011,tag_012,tag_013,tag_014,tag_015,tag_016,tag_017,tag_018,tag_019,tag_020,tag_021,tag_022,tag_023,tag_024,tag_025,tag_026,tag_027,tag_028,tag_029,tag_030,total_price,len
i64,str,str,str,str,str,str,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,u32
1,"""北飯岡ホテル""","""ビジネスホテル""","""岩手県""","""盛岡市""","""北飯岡""","""020-0857""",10300,3.256591,0,0,0,0,0,0,0,0,0,0,0,0,0,1,null,0,0,1,1,0,0,1,null,0,0,1,0,0,0,0,3017900,83
2,"""西二条南温泉ホテル""","""リゾートホテル""","""北海道""","""中川郡美深町""","""西二条南""","""098-2242""",6800,3.453642,0,0,0,null,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,null,1,0,0,0,0,0,0,0
3,"""小屋敷ペンション""","""民宿""","""青森県""","""黒石市""","""小屋敷""","""036-0511""",18600,1.381796,null,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null,0,0,0,1,null,null,0,0,0,0,10248600,143
4,"""中後町民宿""","""民宿""","""愛知県""","""碧南市""","""中後町""","""447-0042""",4900,2.090353,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1808100,94
5,"""鵜沼台ホテル""","""ビジネスホテル""","""岐阜県""","""各務原市""","""鵜沼台""","""509-0121""",9000,2.77149,0,0,0,1,0,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,2421000,73
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
4996,"""田辺通旅館""","""旅館""","""愛知県""","""名古屋市瑞穂区""","""田辺通""","""467-0027""",12100,null,0,null,0,0,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null,0,1,0,6957500,136
4997,"""六本木温泉旅館""","""旅館""","""東京都""","""港区""","""六本木""","""106-8544""",8800,1.499441,null,0,0,0,0,0,0,null,0,0,0,0,0,null,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1663200,57
4998,"""丸山吉野町温泉旅館""","""旅館""","""福岡県""","""北九州市門司区""","""丸山吉野町""","""801-0824""",15700,2.336651,0,0,1,0,0,0,0,0,1,0,0,null,0,0,0,0,null,0,0,0,0,0,0,0,1,0,0,0,null,1,5275200,81
